# SQL Databases
- easy to use from Python
- many different dbs are supported: sqlite3, mysql, postgres, oracle...

In [ ]:
# need some names to build a database, so made some random strings
# but they are hard to read...

import random
import string

string.ascii_lowercase

def makeNames(n):
    # chr takes an ascii code and returns the letter in a string
    n = ''.join([random.choice(string.ascii_lowercase) for j in range(n)])
    return n.capitalize()
    
[makeNames(4) for j in range(10)]

In [ ]:
# nltk corpus has 8,000 real names!
# get some short ones

import random
import nltk

def makeNames(wcnt, wlen):
    names = [w for w in nltk.corpus.names.words() if len(w) == wlen]
    # names are in alphabetical order
    # random doesn't have a 'random subset' routine
    # so do a shuffle 
    random.shuffle(names)
    # then just take random names from the front
    return names[:wcnt]

firsts = makeNames(10, 4)
lasts = makeNames(10, 5)

[firsts, lasts]

In [ ]:
# sqlite stores data in one file
# make a db connection and get a cursor

import sqlite3 
import tempfile

dbf = tempfile.NamedTemporaryFile().name

con = sqlite3.connect(dbf)
cur = con.cursor()
cur

# Create two tables
- grade(uni, course, grade)
- student(first, last, uni)

In [ ]:
cur.execute('create table grade (uni text, course text, grade real)')

In [ ]:
cur.execute('create table student (first text, last text, uni text)')

# Build student table

In [ ]:
# first, last

fl = [ [firsts[k], lasts[k]] for k in range(10)]
fl

In [ ]:
# make an uni
# student = [first, last, uni]

students = [ [f, l, (f[0] + l[0] + str(random.randint(1000,9999))).lower()] 
            for f,l in fl]
students

In [ ]:
# insert above list into db

# 'execute' inserts one row at a time
# expects one row of data
cur.execute('insert into student values(?, ?, ?)', students[0])

# 'executemany' inserts multiple rows at once
# expects list of rows, executes the statement for each row
cur.executemany('insert into student values(?, ?, ?)', students[1:])

'''
# the above two statements are equivalent to 
for student in students:
    cur.execute('insert into student values(?, ?, ?)', student)

# or 
cur.executemany('insert into student values(?, ?, ?)', students)

executemany is typically more efficient 
'''

None

In [ ]:
# the select returns a generator

sg = cur.execute('select first, last, uni from student')
sg

In [ ]:
rows = list(sg)
rows

In [ ]:
# pull the uni out of the tuple 

unis = [t[0] for t in cur.execute('select uni from student')]
unis

In [ ]:
# classes 

classes = ['COMSW {}'.format(random.randint(1000, 9999)) for j in range(5)]
classes

In [ ]:
# students take finals

grades = [ [u , random.choice(classes), 70 + 30 * random.random() ] for u in unis ]
grades

In [ ]:
# executemany inserts multiple rows at once 

cur.executemany('insert into grade values(?, ?, ?)', grades)

In [ ]:
# join the two tables on the uni field

list(cur.execute('select first,last,student.uni, \
                 grade from student,grade where student.uni = grade.uni'))

In [ ]:
# add a filter term

q = 'select last,first,grade from student,grade \
 where grade>80 and student.uni = grade.uni \
 order by last'
list(cur.execute(q))

In [ ]:
# always commit and close the connection

con.commit()
con.close()

In [ ]:
# data is persisted on disk, can read again
# normally would use a with statement to automatically close

res=None

with sqlite3.connect(dbf) as con:
    cur = con.cursor()
    res = list(cur.execute(q))

res

# Object Relational Mappers
- maps objects into a relational database
- somewhat complex but very useful
- best known one is [SQLAlchemy](http://www.sqlalchemy.org)
- [peewee](http://docs.peewee-orm.com/en/latest/) is a simple one
- [hibernate](http://hibernate.org) was a pioneering and hugely successful ORM for Java

# NoSQL databases
- provide less functionality than SQL, but are more efficient and scale better
- [mongodb](https://www.mongodb.org) is a popular one
- [PyMongo](https://docs.mongodb.org/getting-started/python/client/) is the python driver for mongodb
